In [ ]:
from data.

In [1]:
import pandas as pd
import datetime


In [2]:
df_cards = pd.read_csv('df_cards.csv')
df_clients = pd.read_csv('df_clients.csv')
df_transactions = pd.read_csv('transactions_data.csv')

In [3]:
df_cards["expires"] = pd.to_datetime(df_cards["expires"], format='%m/%Y')
df_cards['credit_limit'] = df_cards['credit_limit'].str.replace("$","").astype(float)

In [11]:
df_transactions["date"] = pd.to_datetime(df_transactions["date"], format='%Y-%m-%d %H:%M:%S')

In [4]:
# query_1: The card_id with the latest expiry date and the lowest credit limit amount.


df_cards.sort_values(by=['expires','credit_limit'],ascending=[False,True]).head(5)

,card_id,client_id,acct_open_date,card_brand,card_number,card_on_dark_web,card_type,credit_limit,cvv,expires,has_chip,num_cards_issued,year_pin_last_changed
2235,4490,171,09/1996,Mastercard,5648056200144957,No,Debit (Prepaid),46.0,793,2024-12-01,YES,2,2007
2706,306,1873,01/2012,Visa,4329819283337925,No,Debit (Prepaid),60.0,529,2024-12-01,NO,2,2012
2969,999,1498,02/2002,Mastercard,5909422442877722,No,Debit (Prepaid),65.0,904,2024-12-01,YES,2,2008
3206,4085,1985,08/1996,Mastercard,5461658879915684,No,Debit,625.0,595,2024-12-01,YES,2,2014
538,4292,249,08/2009,Mastercard,5806288548928072,No,Credit,1500.0,798,2024-12-01,YES,2,2009


In [30]:
# query_2: The client_id that will retire within a year that has the lowest credit score and highest debt.

df_clients_wk = df_clients.copy()
df_clients_wk['total_debt']     = df_clients_wk['total_debt'].str.replace('$','').astype(float)
df_clients_wk["retirement_age"] = df_clients_wk["retirement_age"].astype(int)
df_clients_wk["credit_score"]   = df_clients_wk["credit_score"].astype(int)
df_clients_wk["birth_year"]     = df_clients_wk["birth_year"].astype(int)
df_clients_wk["birth_month"]    = df_clients_wk["birth_month"].astype(int)
df_clients_wk["birth"] = pd.to_datetime(df_clients_wk["birth_year"].astype(str) + df_clients_wk["birth_month"].astype(str), format='%Y%m')

df_clients_wk['age'] = (datetime.datetime.now() - df_clients_wk['birth']).dt.days // 365

df_clients_wk["year_to_retirement"] = df_clients["retirement_age"] - df_clients_wk["age"]

#
df_clients_wk.query("year_to_retirement == 1").sort_values(by=['credit_score','total_debt'],
                                                           ascending=[True,False]).head(5)

,client_id,address,birth_month,birth_year,credit_score,current_age,gender,latitude,longitude,num_credit_cards,per_capita_income,retirement_age,total_debt,yearly_income,birth,age,year_to_retirement
802,630,1583 Grant Avenue,7,1959,491,60,Male,30.33,-81.65,3,$19382,66,61861.0,$39521,1959-07-01,65,1
588,1791,4975 Madison Boulevard,11,1958,593,61,Male,39.77,-86.14,3,$16045,67,46919.0,$32716,1958-11-01,66,1
110,1540,1513 Valley Boulevard,9,1964,625,55,Female,39.33,-84.54,3,$21992,61,63194.0,$44839,1964-09-01,60,1
89,872,96 Lake Lane,1,1959,655,61,Female,41.88,-87.84,4,$15451,66,59730.0,$31505,1959-01-01,65,1
23,1029,9218 Martin Luther King Avenue,3,1953,676,66,Male,27.90,-81.97,3,$17944,72,86573.0,$36589,1953-03-01,71,1


In [15]:
# query_3: The transaction_id of an Online purchase on a 31st of December with the highest absolute amount (either earnings or expenses).
df_transactions_wk = df_transactions.copy()
df_transactions_wk["amount"] = df_transactions_wk["amount"].str.replace("$","").astype(float)
df_transactions_wk["amount_abs"] = df_transactions_wk["amount"].abs()

df_transactions_wk.query("merchant_city == 'ONLINE' and date.dt.month == 12 and date.dt.day == 31").sort_values(by='amount_abs',ascending=False).head(5)


,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,amount_abs
2529289,10534178,2011-12-31 09:14:00,1102,2822,914.97,Online Transaction,11722,ONLINE,NaN,NaN,6300,NaN,914.97
10749823,20603233,2017-12-31 14:59:00,1261,4121,912.51,Online Transaction,9932,ONLINE,NaN,NaN,5311,NaN,912.51
3853298,12147901,2012-12-31 23:20:00,371,5495,812.70,Online Transaction,50292,ONLINE,NaN,NaN,7801,NaN,812.70
6570094,15469283,2014-12-31 14:03:00,1427,3946,706.96,Online Transaction,23446,ONLINE,NaN,NaN,7801,NaN,706.96
7957081,17169280,2015-12-31 09:11:00,1385,5568,540.60,Online Transaction,76721,ONLINE,NaN,NaN,6300,NaN,540.60


In [22]:
# query_4: Which client over the age of 40 made the most transactions with a Visa card in February 2016? Please return the client_id, the card_id involved, and the total number of transactions.

id_cards = df_cards[df_cards["card_brand"] == "Visa"]["card_id"].values


df_client = df_clients_wk[df_clients_wk['age']>40]
id_client = df_clients_wk['client_id'].tolist()

df_Feb_2016_Age40_Visa = df_transactions_wk.query("card_id in @id_cards and date.dt.year == 2016 and date.dt.month == 2 and client_id in @id_client")

In [24]:
# count of transactions by client_id and card_id
df_Feb_2016_Age40_count = df_Feb_2016_Age40_Visa.groupby(['client_id', 'card_id']).size().reset_index(name='count')

In [31]:

df_Feb_2016_Age40_count.sort_values(by='count', ascending=False).head(5)

,client_id,card_id,count
370,598,2040,172
1132,1776,3596,160
1240,1954,4879,157
313,476,1070,152
181,254,1247,148
